In [1]:
import os

In [2]:
class Box:
    def __init__(self, name, conf, x1, y1, x2, y2):
        self.name = name
        self.conf = float(conf)
        self.x1 = float(x1)
        self.y1 = float(y1)
        self.x2 = float(x2)
        self.y2 = float(y2)

    def size2list(self):
        return [self.x1, self.y1, self.x2, self.y2]

def parse_txt(filepath, isGt):
    boxes = []
    method = None
    delay = None
    with open(filepath, "r") as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
        if not isGt:
            method, delay = lines[0].split()
            delay = float(delay)
            lines = lines[1:]

        for line in lines:
            split_line = line.split()
            boxes.append(Box(*split_line))
            
    return method, delay, boxes

In [3]:
def bbox_iou(box1, box2, x1y1x2y2=True):
    if x1y1x2y2:
        mx = min(box1[0], box2[0])
        Mx = max(box1[2], box2[2])
        my = min(box1[1], box2[1])
        My = max(box1[3], box2[3])
        w1 = box1[2] - box1[0]
        h1 = box1[3] - box1[1]
        w2 = box2[2] - box2[0]
        h2 = box2[3] - box2[1]
    else:
        w1 = box1[2]
        h1 = box1[3]
        w2 = box2[2]
        h2 = box2[3]

        mx = min(box1[0], box2[0])
        Mx = max(box1[0] + w1, box2[0] + w2)
        my = min(box1[1], box2[1])
        My = max(box1[1] + h1, box2[1] + h2)
    uw = Mx - mx
    uh = My - my
    cw = w1 + w2 - uw
    ch = h1 + h2 - uh
    carea = 0
    if cw <= 0 or ch <= 0:
        return 0.0

    area1 = w1 * h1
    area2 = w2 * h2
    carea = cw * ch
    uarea = area1 + area2 - carea
    return carea / uarea

In [4]:
def get_performance(result_dir, ground_true_dir="./gt"):
    track_sum = 0
    track_delay_sum = 0
    detect_sum = 0
    detect_delay_sum = 0
    track_scores_sum = 0
    detect_scores_sum = 0
    object_sum = 0
    object_loss = 0

    for _, _, files in os.walk(ground_true_dir):
        frame_count = len(files)
        for filename in files:
            gt_path = os.path.join(ground_true_dir, filename)
            result_path = os.path.join(result_dir, filename)
            method, delay, result_boxes = parse_txt(result_path, False)
            _, _, gt_boxes = parse_txt(gt_path, True)
            object_sum += len(gt_boxes)
            if method == "detect":
                detect_sum += 1
                detect_delay_sum += delay / 1e9
                detect_scores_sum += 100 
            else:
                track_sum += 1
                track_delay_sum += delay / 1e9
                temp_scores_sum = 0
                object_loss += len(gt_boxes) - len(result_boxes)
                for result_box in result_boxes:
                    max_iou = 0
                    for gt_box in gt_boxes:
                        iou = bbox_iou(result_box.size2list(), gt_box.size2list())
                        if iou > max_iou:
                            max_iou = iou
                    if max_iou > 0.9:
                        score = 100
                    elif max_iou > 0.8:
                        score = 95
                    elif max_iou > 0.7:
                        score = 90
                    elif max_iou > 0.6:
                        score = 80
                    elif max_iou > 0.5:
                        score = 70
                    else:
                        score = max_iou * 100
                        object_loss += 1
                    temp_scores_sum += score
                track_scores_sum += temp_scores_sum / (len(gt_boxes) * 100) * 100
    detect_delay_avg = detect_delay_sum / detect_sum
    track_delay_avg = track_delay_sum / track_sum
    return detect_delay_avg, detect_delay_sum, track_delay_avg, track_delay_sum, detect_scores_sum/detect_sum, track_scores_sum/track_sum, object_sum, object_loss, object_loss/object_sum*100

In [8]:
import wcwidth
from tabulate import tabulate
data = []
data.append(["lk500-0.01", *get_performance("./lk500-0.01")])
data.append(["lk1000-0.01", *get_performance("./lk1000-0.01")])
data.append(["lk1000-0.1", *get_performance("./lk1000-0.1")])
data.append(["medainflow", *get_performance("./medainflow")])
data.append(["kcf", *get_performance("./kcf")])
data.append(["mosse", *get_performance("./mosse")])
data.append(["csrt", *get_performance("./csrt")])
data.append(["mil", *get_performance("./mil")])
data.append(["boosting", *get_performance("./boosting")])
data.append(["tld", *get_performance("./tld")])
print(tabulate(data, headers=["method", "detect_delay_avg", "detect_delay_sum", "track_delay_avg", "track_delay_sum", "detect_scores", "track_scores", "object_sum", "object_loss", "object_loss_rate"]))

method         detect_delay_avg    detect_delay_sum    track_delay_avg    track_delay_sum    detect_scores    track_scores    object_sum    object_loss    object_loss_rate
-----------  ------------------  ------------------  -----------------  -----------------  ---------------  --------------  ------------  -------------  ------------------
lk500-0.01            0.01698               1.30746         0.0122775            0.945369              100         79.7388          1646            139             8.44471
lk1000-0.01           0.0170399             1.31207         0.0151333            1.16526               100         81.2409          1646            124             7.53341
lk1000-0.1            0.0173345             1.33475         0.0114835            0.884231              100         79.3707          1646            149             9.05225
medainflow            0.0268482             2.06731         0.0179665            1.38342               100         79.7186          1646    